In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skfeature.function.information_theoretical_based import MRMR

pd.set_option('display.max_columns', None)

In [2]:
signal = pd.read_csv("signal.csv", sep = ";")
bkg = pd.read_csv("background.csv", sep = ";")


In [3]:
signal = signal.drop(signal.filter(regex='MC').columns, axis=1)
signal = signal.drop(signal.filter(regex='Weight').columns, axis=1)
signal = signal.drop(signal.filter(regex='Corsika').columns, axis=1)
signal = signal.drop(signal.filter(regex='end').columns, axis=1)
signal = signal.drop(signal.filter(regex='start').columns, axis=1)
signal = signal.drop(signal.filter(regex='time').columns, axis=1)
signal = signal.drop('label', axis=1)

In [4]:
bkg = bkg.drop(bkg.filter(regex='MC').columns, axis=1)
bkg = bkg.drop(bkg.filter(regex='Weight').columns, axis=1)
bkg = bkg.drop(bkg.filter(regex='Corsika').columns, axis=1)
bkg = bkg.drop(bkg.filter(regex='end').columns, axis=1)
bkg = bkg.drop(bkg.filter(regex='start').columns, axis=1)
bkg = bkg.drop(bkg.filter(regex='time').columns, axis=1)
bkg = bkg.drop('label', axis=1)

In [5]:
signal.replace([np.inf, -np.inf], np.nan)
signal.dropna(axis = 'columns', inplace = True)
signal = signal.drop(signal.std()[(signal.std() == 0)].index, axis=1)
#signal.dropna(inplace = True)

bkg.replace([np.inf, -np.inf], np.nan)
bkg.dropna(axis = 'columns', inplace = True)
bkg = bkg.drop(bkg.std()[(bkg.std() == 0)].index, axis=1)
#bkg.dropna(inplace = True)

In [6]:
bcol = bkg.columns
scol = signal.columns

In [7]:
for att in scol:
    if att not in bcol:
        signal.drop(att, axis=1, inplace = True)
    
for att in bcol:
    if att not in scol:
        bkg.drop(att, axis=1, inplace = True)

In [8]:
len(signal.columns)

118

In [9]:
len(bkg.columns)

118

In [10]:
#removed_scol = [j for i, j in zip(signal.columns, bkg.columns) if i != j]
#removed_scol

In [11]:
import scipy.io
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn import svm

In [12]:
sig_label = np.zeros(signal.shape[0])
bkg_label = np.ones(bkg.shape[0])

In [13]:
combined_df = pd.concat([signal, bkg], ignore_index=True)
combined_label = np.append(sig_label, bkg_label)

In [14]:
combined_df.insert(118, 'label', combined_label)
shuffled = combined_df.sample(frac = 1)

In [15]:
y = shuffled['label']
X = shuffled.drop('label', axis=1)

In [16]:
from sklearn import (
    ensemble, linear_model, neighbors, svm, tree, naive_bayes,
    gaussian_process, neural_network, dummy)
from sklearn.model_selection import KFold
from sklearn.base import clone
from tqdm import tqdm
model = ensemble.RandomForestRegressor(n_estimators=100)

In [17]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
X_new = SelectKBest(score_func=mutual_info_classif, k=20)
data_selected = X_new.fit_transform(X, y)
data_selec_cols = X.columns.values[X_new.get_support()]
df = pd.DataFrame(data_selected, columns = data_selec_cols)

In [ ]:
cv = KFold(5, shuffle=True, random_state=1337)

from sklearn.metrics import accuracy_score
from pymrmre import mrmr
models = []
predictions = []
true_values = []

#n_feat = 10

for train_index, test_index in cv.split(df):
    X_train, X_test = df.iloc[train_index, :], df.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    t_val = y_test
    
    predictions.append(pred)
    true_values.append(t_val)
    models.append(model)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import r2_score
for i in range(5):
    r2 = r2_score(predictions[i], true_values[i])
    print(r2)